In [24]:
!pip install dgl

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 59.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 6.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 20.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 19.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 KB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 12.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 7.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
zfit 0.20.3 requires colorama, which is not installed.
zfit 0.20.3 requires colored, which is not installed.
zfit 0.20.3 requires frozendict, which is not installed.
zfit 0.20.3 requires jacobi, which is not installed.
zfit 0.20.3 requires numdifftools, which is not installed.
zfit 0.20.3 requires ordered-set, which is not installed.
zfit 0.20.3 requires texttable, which is not installed.
zfit 0.20.3 requires zfit-interface, which is not installed.
tensorflow 2.12.0 requires gast<=0.4.0,>=0.2.1, but you have gast 0.5.2 which is incompatible.
zfit 0.20.3 requires tensorflow<2.17,>=2.16, but you have tensorflow 2.12.0 which is incompatible.
ydata-profiling 4.6.5 requires pydantic>=2, but you have pydan

In [40]:
!pip install dgl==0.6.1


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 25.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: dgl
    Found existing installation: dgl 2.1.0
    Uninstalling dgl-2.1.0:
      Successfully uninstalled dgl-2.1.0


In [14]:
import uproot
import numpy as np
import awkward as ak
import torch
import sys
sys.path.append('/eos/home-i02/h/hsharma/.local/lib/python3.9/site-packages')

sys.path.append('/eos/user/h/hsharma/.local/bin')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Open the ROOT file and tree
file = uproot.open("/eos/user/h/hsharma/PythiaFIles/Cascades_Xi/cascade_data.root")
tree = file["cascade_tree"]

# Get all branches as an awkward array
data = tree.arrays(library="ak")

# Select only the first 1000 entries
data = data[:1000]  # No need to call `.array()` afterward

# Extract relevant variables
px = data["px"]
py = data["py"]
pz = data["pz"]
charge = data["charge"]

# Convert to NumPy arrays for further processing
px = ak.to_numpy(px)
py = ak.to_numpy(py)
pz = ak.to_numpy(pz)
charge = ak.to_numpy(charge)

# Create a node feature matrix
node_features = np.column_stack((px, py, pz, charge))

print("Node feature matrix shape:", node_features.shape)


Node feature matrix shape: (1000, 4)


In [3]:
np.size(node_features)



4000

In [12]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Number of particles (nodes) in the event
num_particles = len(node_features)

# Create edges (connect all pairs of particles)
edges = []
for i in range(num_particles):
    for j in range(i + 1, num_particles):
        edges.append([i, j])  # Directed edge from i to j
        edges.append([j, i])  # Undirected edge: back from j to i

# Convert edges to tensor format for DGL
edges = torch.tensor(edges, dtype=torch.long).t().contiguous()


In [13]:
import torch
import torch.nn as nn
import dgl
from dgl.nn import GraphConv

class GNNModel(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super(GNNModel, self).__init__()
        # Define the layers of the GNN
        self.conv1 = GraphConv(in_feats, hidden_feats)
        self.conv2 = GraphConv(hidden_feats, out_feats)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, g):
        # Perform graph convolution
        x = g.ndata['features']  # Node features
        x = self.conv1(g, x)  # First graph convolution layer
        x = self.relu(x)
        x = self.conv2(g, x)  # Second graph convolution layer
        x = self.softmax(x)   # Apply softmax for classification output
        return x


In [9]:
# Define the model
model = GNNModel(in_feats=4, hidden_feats=64, out_feats=2)
num_epochs = 50  # Define the number of epochs for training

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(graph)
    loss = loss_fn(output, graph.ndata['label'])
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")


KeyError: 'features'

In [7]:
threshold_distance = 0.5  # Example threshold for proximity

# Create edges based on proximity (this is just a simple example, you may want to use a more sophisticated approach)
edges = []
for i in range(num_particles):
    for j in range(i + 1, num_particles):
        distance = np.sqrt((px[i] - px[j])**2 + (py[i] - py[j])**2 + (pz[i] - pz[j])**2)
        if distance < threshold_distance:
            edges.append([i, j])  # Directed edge from i to j
            edges.append([j, i])  # Undirected edge: back from j to i

# Convert edges to tensor format for DGL
edges = torch.tensor(edges, dtype=torch.long).t().contiguous()

# Rebuild the graph with the new edges
graph = dgl.graph((edges[0], edges[1]), num_nodes=num_particles)
graph.ndata['features'] = node_features_tensor


NameError: name 'node_features_tensor' is not defined

In [8]:
import ROOT
import torch
from torch_geometric.data import Data

# Open the ROOT file
# root_file = ROOT.TFile("cascade_data.root", "READ")
# tree = root_file.Get("cascade_tree")

# Initialize list to store graph data
graphs = []

# Loop through ROOT entries (events)
for entry in range(tree.GetEntries()):
    tree.GetEntry(entry)
    
    # Extract track features (nodes)
    nodes = []
    for i in range(tree.GetEntries()):
        nodes.append([tree.px[i], tree.py[i], tree.pz[i], tree.pT[i], tree.eta[i], tree.phi[i], tree.charge[i], tree.pid[i]])

    # Convert to tensor (PyTorch)
    node_features = torch.tensor(nodes, dtype=torch.float)

    # Construct edges (example: connect tracks from the same event)
    edge_index = []
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            # Add edges if tracks are in close proximity (decay candidate)
            if some_condition:  # Define a condition, e.g., invariant mass, DCA
                edge_index.append([i, j])
                edge_index.append([j, i])

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    # Example target (1 for Xi, 0 for non-cascade decay)
    target = 1 if tree.isXi[0] == 1 else 0
    
    # Create a graph data object
    graph_data = Data(x=node_features, edge_index=edge_index, y=torch.tensor([target], dtype=torch.float))
    graphs.append(graph_data)

# Now `graphs` holds all the graph data for training


Welcome to JupyROOT 6.28/06


AttributeError: 'Model_TTree_v20' object has no attribute 'GetEntries'

In [54]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class GNNModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))  # Graph convolution layer 1
        x = F.dropout(x, training=self.training)  # Dropout to prevent overfitting
        x = self.conv2(x, edge_index)  # Graph convolution layer 2
        return F.log_softmax(x, dim=1)  # Log softmax for classification

# Define the model
model = GNNModel(input_dim=8, hidden_dim=64, output_dim=2)  # Example input/output dims

# Set up optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()


In [55]:
from torch_geometric.loader import DataLoader

# Split your data into training and testing sets
train_loader = DataLoader(graphs[:int(0.8*len(graphs))], batch_size=32, shuffle=True)
test_loader = DataLoader(graphs[int(0.8*len(graphs)):], batch_size=32, shuffle=False)

# Training loop
for epoch in range(10):  # 10 epochs, adjust as needed
    model.train()
    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)  # Compare predicted and true labels
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

# Evaluation loop
model.eval()
correct = 0
total = 0
for data in test_loader:
    out = model(data)
    pred = out.argmax(dim=1)
    correct += (pred == data.y).sum().item()
    total += data.y.size(0)

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")


NameError: name 'graphs' is not defined